In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain_groq import ChatGroq #groq install


In [4]:
#load dot env
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
KEY= os.getenv("GROQ_API_KEY")

In [6]:
llm = ChatGroq(temperature=0.5, groq_api_key=KEY, model_name="mixtral-8x7b-32768")

In [7]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000202853274C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020285326800>, temperature=0.5, groq_api_key=SecretStr('**********'))

In [8]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [10]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [11]:
#prompt template
quiz_generation_prmpt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [12]:
#llm chain1
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prmpt,output_key="quiz",verbose=True)

c:\McqGen\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [13]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [14]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [15]:
#llm chain 2
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [16]:
#combine two chains
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [17]:
file_path=r"C:\McqGen\data.txt"

In [18]:
file_path

'C:\\McqGen\\data.txt'

In [19]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [20]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [21]:
#serialize the python dictionary into a json-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [22]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"

In [23]:
#tracking token
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

c:\McqGen\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used b

In [24]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")


Total Tokens:2106
Prompt Tokens:1542


In [25]:
quiz=response.get("quiz")

In [26]:
quiz=json.loads(quiz)

In [27]:
quiz_table_data=[]
for key, value in quiz.items():
    mcq = value["mcq"]
    options= " | ".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct= value["correct"]
    quiz_table_data.append({"MCQ":mcq, "Choices":options, "Correct": correct})

In [28]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning' in 1959?",
  'Choices': 'a:Arthur Samuel | b:Raytheon Company | c:Donald Hebb | d:Warren McCulloch',
  'Correct': 'a'},
 {'MCQ': "What did Donald Hebb introduce in his book 'The Organization of Behavior' in 1949?",
  'Choices': 'a:Theoretical neural structure | b:Reinforcement learning | c:Artificial neurons | d:Pattern classification',
  'Correct': 'a'},
 {'MCQ': "What was the main focus of Nilsson's book 'Learning Machines' in the 1960s?",
  'Choices': 'a:Pattern recognition | b:Neural networks | c:Reinforcement learning | d:Machine learning for pattern classification',
  'Correct': 'd'},
 {'MCQ': 'What was the report given in 1981 about?',
  'Choices': 'a:Using teaching strategies for artificial neural networks | b:Inventing a program that calculated the winning chance in checkers | c:Analyzing sonar signals, electrocardiograms, and speech patterns | d:Classifying cancerous moles using computer vision',
  'Correct': 'a'},
 {'MCQ': 'Wh

In [29]:
quiz=pd.DataFrame(quiz_table_data)

In [30]:
quiz.to_csv("MachineLearning.csv", index=False)